In [6]:
df_QUERY_notasExport = pd.read_excel(
    'uploads//nota_export.xlsx')

In [4]:
from flask import Flask, render_template, send_from_directory, request, redirect, url_for, Response
import os
from zipfile import ZipFile
import shutil
import subprocess
from werkzeug.utils import secure_filename
import pandas as pd
import numpy as np
import time
from datetime import datetime


In [7]:
bpk = df_QUERY_notasExport

df_QUERY_notasExport_filtred = df_QUERY_notasExport[(df_QUERY_notasExport['Status do sistema'] == 'MSPN') | (df_QUERY_notasExport['Status do sistema'] == 'MSPR')]
df_QUERY_notasExport_filtred

,Data da nota,Hora da nota,Equipam.,Descrição,TpNota,Prioridade,Parada,Notificador,Texto do item,Texto da causa,CódFalha,Sub-Trecho,Nota,Status do sistema,Centro,Centro trab.respons.
0,2024-11-26,07:16:17,0313424HPT,FR|EG|SUPORTE DDV|QUEBRADO,M1,1.0,NaN,RICHARD,*DDV:Suporte*,||||||KM 0092+000 TREM MNB,VGDV,PCZ-ZPG,23906113,MSPN,NaN,NaN
1,2024-11-26,08:34:59,0318302HPT,SE|LONG|BAT DIANTEIRO|TRINCA<200|60mm,M1,1.0,NaN,RICARDO,*Longarina:Trinca*,|CA-LD|60 mm|||LONG BAT DIANTEIRO TRINCA,VGLO,PCZ-ZPG,23906188,MSPN,NaN,NaN
2,2024-11-26,07:59:25,0333921HPT,TR|AB|PARAF DA CHAPA AB|QUEBRADO| 1,M1,1.0,NaN,FLAVIANE,*Amort:Chapa AB*,|CB-LD|||1 qtd|AB PARAF DA CHAPA AB QUEB,VGAB,PCZ-ZPG,23906068,MSPN,NaN,NaN
3,2024-11-26,17:42:18,8403597HPT,CCT|ENGATE|HASTE MANOBRA|FALT|,M1,1.0,NaN,RICARDO,*CCT:Haste de Manobra*,|CB|||| ENGATE HASTE MANOBRA FALT,VGJV,PCZ-ZPG,23827005,MSPN,NaN,NaN
4,2024-11-26,08:37:32,8404585HPT,TR|MOLA|MOLA MARCA SOLIDA |,M1,1.0,NaN,FLAVIANE,*Amort:Mola*,CA LD interna mola marca solida,VGTM,ZAR-ZTO,23906189,MSPN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10887,2024-06-01,03:42:54,8401322HPT,REVI MONITORADO,M1,1.0,NaN,FLAVIANE,*Amort:Mola*,CB le 2 molas carga solida,VGTM,PCZ-ZPG,23767953,MSPN,NaN,NaN
10888,2024-06-01,19:48:23,8413304HFT,TR|PACOTE DE MOLA|MOLA|MARCA SOLIDA,M1,1.0,NaN,SHEILLA,*Amort:Mola*,CB L/E 1 mola carga sólida,VGTM,PCZ-ZPG,23768120,MSPN,NaN,NaN
10889,2024-06-01,21:41:58,0559288HFT,REVI MONITORAMENTO,M1,1.0,NaN,SHEILLA,*Rd:Pista*,R4 LD escoamento de material,VGHB,PCZ-ZPG,23768157,MSPN,NaN,NaN
10890,2024-06-01,04:59:14,8417628HFT,KM 000+000 TREM I48,M1,3.0,NaN,KEROLLYN,Não aplica freio - roda fria,Roda fria (CW),VGFH,NaN,23767939,MSPN,NaN,NaN


In [8]:
    print("inicio")
    global user_info
    global df_stage_merge
    # Carregar o arquivo JSON em um DataFrame
    #    df_QUERY_z369 = pd.read_json(
    #        '\\\\fs-vwps02.all-logistica.net\\GRUPOS2\\CIM\\Coordenação de monitoramento de Material Rodante\\Data_STAGE\\df_stage_Nota.json', lines=True)
    #    df_QUERY_TELA_164_FOTO = pd.read_json(
    #        '\\\\fs-vwps02.all-logistica.net\\GRUPOS2\\CIM\\Coordenação de monitoramento de Material Rodante\\Data_STAGE\\df_QUERY_TELA_164_FOTO.json', lines=True)
    print("leituradf_QUERY_z369")
    df_QUERY_z369 = pd.read_json(
        'files\df_stage_Nota.json', lines=True)
    print("leituradf_notasExport")
    df_QUERY_notasExport = pd.read_excel(
        'uploads//nota_export.xlsx')
    print("leituradf_QUERY_TELA_164_FOTO")
    df_QUERY_TELA_164_FOTO = pd.read_json(
        'files\df_QUERY_TELA_164_FOTO.json', lines=True)

    if len(df_QUERY_notasExport) > 1:
        print("nota_export OK ")
        # Obter os nomes atuais das colunas
        colunas_atuais = df_QUERY_notasExport.columns
        df_QUERY_notasExport_filtred = df_QUERY_notasExport[(df_QUERY_notasExport['Status do sistema'] == 'MSPN') | (
            df_QUERY_notasExport['Status do sistema'] == 'MSPR')]

        # Mapeamento entre nomes antigos e novos
        mapa_colunas = {
            'Data da nota': 'DT_NOTA',
            'Hora da nota': 'Hora da nota',
            'Equipam.': 'ATIVO_TL',
            'Parada': 'Flag',
            'Notificador': 'Notificador',
            'TpNota': 'TP_NOTA',
            'CódFalha': 'CodFalha',
            'Descrição': 'Texto_AVARIA',
            'Texto do item': 'Texto_Item',
            'Texto da causa': 'Texto_Causa',
            'Status do sistema': 'Aux_Status',
            'Nota': 'NOTA',
            'Novo Descrição': 'TEXTO',
            'Novo Texto do item': 'Texto_Item',
            'Novo Texto da causa': 'Texto_Causa',
            'Modelo': 'ATIVO_TL'
        }

        # Criar uma lista de novos nomes, mantendo os originais onde não há correspondência
        colunas_convertidas = [mapa_colunas.get(
            col, col) for col in colunas_atuais]

        # Garantir que o número de nomes no mapeamento corresponda ao número de colunas
        if len(colunas_convertidas) == len(colunas_atuais):
            # Renomear as colunas no DataFrame `df_QUERY_notasExport`
            df_QUERY_notasExport_filtred.columns = colunas_convertidas
        else:
            print(
                "Erro: O número de colunas mapeadas não corresponde ao número de colunas no DataFrame.")

        df_QUERY_notasExport_filtred['BASE'] = "SAP_Arquivo_LOCAL"
        df_QUERY_z369['BASE'] = "SAP_STAGE"

        df_concatenado = pd.concat(
            [ df_QUERY_z369,df_QUERY_notasExport_filtred])
        
        df_resultado_Export_z369_final = df_concatenado.drop_duplicates(
            subset=['NOTA'], keep='last')


    else:
        user_info = "Sem dados Locais - APENAS STAGE"
        print("nota_export NOK")

    if len(df_QUERY_TELA_164_FOTO) > 1000:
        print("df_QUERY_TELA_164_FOTO OK ")
    else:
        print("df_QUERY_TELA_164_FOTO NOK ")

    if len(df_QUERY_z369) > 1000:
        print("df_QUERY_z369 OK ")
    else:
        print("df_QUERY_z369 NOK ")

    # ponto chave para textes
    df_QUERY_z369 = df_resultado_Export_z369_final
    df_QUERY_z369_bkp = df_resultado_Export_z369_final

    # ponto chave para textes

    df_QUERY_TELA_164_FOTO["ATIVO_TL"] = df_QUERY_TELA_164_FOTO["VAGAO"].astype(
        str) + df_QUERY_TELA_164_FOTO["SERIE"].astype(str)
    df_stage_merge = pd.merge(df_QUERY_TELA_164_FOTO,
                              df_QUERY_z369, on='ATIVO_TL', how='left')
    df_stage_merge['NR_OS'] = df_stage_merge['NR_OS'].apply(
        lambda x: str(int(x)) if pd.notnull(x) else '')

    # Pontução Trem
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.max_colwidth')
    Pontuacao_trem = pd.pivot_table(
        df_stage_merge, index='NR_OS', columns='TP_NOTA', aggfunc='size', fill_value=0)
    Pontuacao_trem = Pontuacao_trem.reset_index()
    Pontuacao_trem = Pontuacao_trem.merge(
        df_stage_merge[['TREM', 'NR_OS']], on='NR_OS', how='left')
    Pontuacao_trem = Pontuacao_trem.sort_values(
        by='M2', ascending=False).drop_duplicates().reset_index()
    Pontuacao_trem = Pontuacao_trem[['TREM',	'M1',	'M2',	'NR_OS']]
    Pontuacao_trem['TREM'] = Pontuacao_trem['TREM'].astype(str)
    Pontuacao_trem = Pontuacao_trem[(Pontuacao_trem['TREM'] != "None")]

    df = Pontuacao_trem

    df_QUERY_TELA_164_FOTO["ATIVO_TL"] = df_QUERY_TELA_164_FOTO["VAGAO"].astype(
        str) + df_QUERY_TELA_164_FOTO["SERIE"].astype(str)
    df_stage_merge = pd.merge(df_QUERY_TELA_164_FOTO,
                              df_QUERY_z369, on='ATIVO_TL', how='left')
    df_stage_merge['NR_OS'] = df_stage_merge['NR_OS'].apply(
        lambda x: str(int(x)) if pd.notnull(x) else '')

inicio
leituradf_QUERY_z369
leituradf_notasExport
leituradf_QUERY_TELA_164_FOTO
nota_export OK 
df_QUERY_TELA_164_FOTO OK 
df_QUERY_z369 OK 


C:\Users\leona\AppData\Local\Temp\ipykernel_25888\1845013801.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_QUERY_notasExport_filtred['BASE'] = "SAP_Arquivo_LOCAL"


In [ ]:
df_QUERY_z369.groupby('BASE').size()

BASE
SAP_Arquivo_LOCAL    10531
SAP_STAGE             4609
dtype: int64

In [ ]:
df_concatenado.groupby('BASE').size()

BASE
SAP_Arquivo_LOCAL    10532
SAP_STAGE            15090
dtype: int64

In [ ]:
df_concatenado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25622 entries, 0 to 10891
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   NOTA                  25622 non-null  int64  
 1   DT_NOTA               25622 non-null  object 
 2   Notificador           25591 non-null  object 
 3   Texto_Causa           25494 non-null  object 
 4   TP_NOTA               25622 non-null  object 
 5   TEXTO                 15090 non-null  object 
 6   Texto_Item            25600 non-null  object 
 7   Texto_AVARIA          25596 non-null  object 
 8   DATA_QMDAT            15090 non-null  float64
 9   CodFalha              25596 non-null  object 
 10  Flag                  15149 non-null  object 
 11  Aux_Status            25622 non-null  object 
 12  ATIVO_TL              25622 non-null  object 
 13  BASE                  25622 non-null  object 
 14  Hora da nota          10532 non-null  object 
 15  Prioridade         

In [ ]:
df_resultado_Export_z369_final_d.groupby('BASE').size()

BASE
SAP_Arquivo_LOCAL       51
SAP_STAGE            15089
dtype: int64

In [ ]:
df_stage_merge['NOTA'] = df_stage_merge['NOTA'].fillna(0).astype(int)

KeyError: 'Coluna'

In [ ]:
df_QUERY_notasExport

,Data da nota,Hora da nota,Equipam.,Descrição,TpNota,Prioridade,Parada,Notificador,Texto do item,Texto da causa,CódFalha,Sub-Trecho,Nota,Status do sistema,Centro,Centro trab.respons.
0,2024-11-26,07:16:17,0313424HPT,FR|EG|SUPORTE DDV|QUEBRADO,M1,1.0,NaN,RICHARD,*DDV:Suporte*,||||||KM 0092+000 TREM MNB,VGDV,PCZ-ZPG,23906113,MSPN,NaN,NaN
1,2024-11-26,08:34:59,0318302HPT,SE|LONG|BAT DIANTEIRO|TRINCA<200|60mm,M1,1.0,NaN,RICARDO,*Longarina:Trinca*,|CA-LD|60 mm|||LONG BAT DIANTEIRO TRINCA,VGLO,PCZ-ZPG,23906188,MSPN,NaN,NaN
2,2024-11-26,07:59:25,0333921HPT,TR|AB|PARAF DA CHAPA AB|QUEBRADO| 1,M1,1.0,NaN,FLAVIANE,*Amort:Chapa AB*,|CB-LD|||1 qtd|AB PARAF DA CHAPA AB QUEB,VGAB,PCZ-ZPG,23906068,MSPN,NaN,NaN
3,2024-11-26,17:42:18,8403597HPT,CCT|ENGATE|HASTE MANOBRA|FALT|,M1,1.0,NaN,RICARDO,*CCT:Haste de Manobra*,|CB|||| ENGATE HASTE MANOBRA FALT,VGJV,PCZ-ZPG,23827005,MSPN,NaN,NaN
4,2024-11-26,08:37:32,8404585HPT,TR|MOLA|MOLA MARCA SOLIDA |,M1,1.0,NaN,FLAVIANE,*Amort:Mola*,CA LD interna mola marca solida,VGTM,ZAR-ZTO,23906189,MSPN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10887,2024-06-01,03:42:54,8401322HPT,REVI MONITORADO,M1,1.0,NaN,FLAVIANE,*Amort:Mola*,CB le 2 molas carga solida,VGTM,PCZ-ZPG,23767953,MSPN,NaN,NaN
10888,2024-06-01,19:48:23,8413304HFT,TR|PACOTE DE MOLA|MOLA|MARCA SOLIDA,M1,1.0,NaN,SHEILLA,*Amort:Mola*,CB L/E 1 mola carga sólida,VGTM,PCZ-ZPG,23768120,MSPN,NaN,NaN
10889,2024-06-01,21:41:58,0559288HFT,REVI MONITORAMENTO,M1,1.0,NaN,SHEILLA,*Rd:Pista*,R4 LD escoamento de material,VGHB,PCZ-ZPG,23768157,MSPN,NaN,NaN
10890,2024-06-01,04:59:14,8417628HFT,KM 000+000 TREM I48,M1,3.0,NaN,KEROLLYN,Não aplica freio - roda fria,Roda fria (CW),VGFH,NaN,23767939,MSPN,NaN,NaN


In [ ]:
df_stage_merge




,VAGAO,ID_VAGAO,SERIE,FROTA_RESTRITA,FROTA_NAO_RESTRITA,TREM,SEQUENCIA,COD_LINHA,LOCAL,ORIGEM,...,DATA_QMDAT,CodFalha,Flag,Aux_Status,BASE,Hora da nota,Prioridade,Sub-Trecho,Centro,Centro trab.respons.
0,3509885,371606,HTT,NaN,971;973;857;632;626,None,50.0,L4,PSG,PCZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,321575,345310,FTT,NaN,632;721;546;457,L85,16.0,None,ZIZ,ZPG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8408572,329224,HFT,NaN,460;893;891;891;379;632;497;396;505;958,None,33.0,MULTICARGO,PST,---,...,20240930.0,VGAB,,Em processamento,SAP_STAGE,NaN,NaN,NaN,NaN,NaN
3,8408572,329224,HFT,NaN,460;893;891;891;379;632;497;396;505;958,None,33.0,MULTICARGO,PST,---,...,20241122.0,VGCT,,Em processamento,SAP_STAGE,NaN,NaN,NaN,NaN,NaN
4,992194,353592,TCT,NaN,857;128;896,T61,76.0,None,ZQX,ZZZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27865,978906,323839,HFT,NaN,460;893;891;379;895;632,Y84,120.0,None,ZBV,PRV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27866,990655,324320,HFT,NaN,460;893;615;167;379;616;632;989,R72,26.0,None,ZFN,ZFN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27867,3021475,375196,FLT,NaN,632;857,L92,61.0,None,ZGB,TIN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27868,985732,324681,HFT,NaN,460;893;891;167;379;632;615;725;989,I82,64.0,None,ZXW,TRO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_QUERY_notasExport

,Data da nota,Hora da nota,Equipam.,Descrição,TpNota,Prioridade,Parada,Notificador,Texto do item,Texto da causa,CódFalha,Sub-Trecho,Nota,Status do sistema,Centro,Centro trab.respons.
0,2024-11-26,07:16:17,0313424HPT,FR|EG|SUPORTE DDV|QUEBRADO,M1,1.0,NaN,RICHARD,*DDV:Suporte*,||||||KM 0092+000 TREM MNB,VGDV,PCZ-ZPG,23906113,MSPN,NaN,NaN
1,2024-11-26,08:34:59,0318302HPT,SE|LONG|BAT DIANTEIRO|TRINCA<200|60mm,M1,1.0,NaN,RICARDO,*Longarina:Trinca*,|CA-LD|60 mm|||LONG BAT DIANTEIRO TRINCA,VGLO,PCZ-ZPG,23906188,MSPN,NaN,NaN
2,2024-11-26,07:59:25,0333921HPT,TR|AB|PARAF DA CHAPA AB|QUEBRADO| 1,M1,1.0,NaN,FLAVIANE,*Amort:Chapa AB*,|CB-LD|||1 qtd|AB PARAF DA CHAPA AB QUEB,VGAB,PCZ-ZPG,23906068,MSPN,NaN,NaN
3,2024-11-26,17:42:18,8403597HPT,CCT|ENGATE|HASTE MANOBRA|FALT|,M1,1.0,NaN,RICARDO,*CCT:Haste de Manobra*,|CB|||| ENGATE HASTE MANOBRA FALT,VGJV,PCZ-ZPG,23827005,MSPN,NaN,NaN
4,2024-11-26,08:37:32,8404585HPT,TR|MOLA|MOLA MARCA SOLIDA |,M1,1.0,NaN,FLAVIANE,*Amort:Mola*,CA LD interna mola marca solida,VGTM,ZAR-ZTO,23906189,MSPN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10887,2024-06-01,03:42:54,8401322HPT,REVI MONITORADO,M1,1.0,NaN,FLAVIANE,*Amort:Mola*,CB le 2 molas carga solida,VGTM,PCZ-ZPG,23767953,MSPN,NaN,NaN
10888,2024-06-01,19:48:23,8413304HFT,TR|PACOTE DE MOLA|MOLA|MARCA SOLIDA,M1,1.0,NaN,SHEILLA,*Amort:Mola*,CB L/E 1 mola carga sólida,VGTM,PCZ-ZPG,23768120,MSPN,NaN,NaN
10889,2024-06-01,21:41:58,0559288HFT,REVI MONITORAMENTO,M1,1.0,NaN,SHEILLA,*Rd:Pista*,R4 LD escoamento de material,VGHB,PCZ-ZPG,23768157,MSPN,NaN,NaN
10890,2024-06-01,04:59:14,8417628HFT,KM 000+000 TREM I48,M1,3.0,NaN,KEROLLYN,Não aplica freio - roda fria,Roda fria (CW),VGFH,NaN,23767939,MSPN,NaN,NaN
